In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

# for visualisation 

In [ ]:
# Load the data there

In [ ]:
# This func converts both 3d and 2d np arrays (chunc_axis,time_channel,_axis) and (chunc_axis, class_axis), to the DF with multilevel index
def create_multidim_timeseries_df(array_X, array_y):
    names = ['chunk', 'time', 'sensor']
    index = pd.MultiIndex.from_product([range(s)for s in array_X.shape], names=names)
    X_df = pd.DataFrame({'Train': array_X.flatten()}, index=index)['Train']
    y_df = pd.DataFrame(data = np.argmax(array_y, axis=1), columns=['class'])
    y_df.index.name = 'chunk'
    df = pd.merge(X_df, y_df, right_index=True, left_on=['chunk'])  
    df = pd.pivot_table(df.reset_index(), values='Train', index=['class','chunk', 'time'], columns=['sensor']) # relativly slow row
    df = df.sort_index(level=1) # set level
    return df

train = create_multidim_timeseries_df(X_train_raw, y_train_raw)
test = create_multidim_timeseries_df(X_test_raw, y_test_raw)

In [ ]:
# let's build the diagram of the distribution of measurements by the class
# observe the relative balance of the classes 
temp_df = train.reset_index()
class_counts = temp_df['class'].value_counts()

sns.barplot(x=temp_df['class'].value_counts().index, y=temp_df['class'].value_counts()/ X_train.shape[1])
plt.title(f"Class distribution")

In [ ]:
# visualize the distribution density depending on the classes within each sensor channel.

train[0].reset_index().drop(columns=['time','chunk'])

temp_df = train.reset_index().drop(columns=['time','chunk']) 
for sensor_ch, col in enumerate(temp_df.columns[1:]):
    plt.figure(sensor_ch)
    sns.kdeplot(data= temp_df, x= sensor_ch, hue='class',  palette='bright')
    plt.title(f"Channel {sensor_ch}")

In [ ]:
# let's build plots with timeseries different classes  with confedence interval (chunks projection)
temp_df2 = train.reset_index()
for class_id in range(len(temp_df2['class'].unique())):  
    temp_df3 = temp_df2[temp_df2['class'] == class_id].drop(columns=['class','chunk'])
    print(temp_df3.columns[1:])
    temp_df3 = pd.melt(temp_df3, id_vars=['time'], value_vars=temp_df3.columns[1:])
    plt.figure(class_id)
    sns.lineplot(x="time", y="value", hue="sensor", data=temp_df3,  palette='bright')
    plt.title(f"Class {class_id}")